In [42]:
import os
import csv
import pandas as pd
import numpy as np
from IPython.display import clear_output
import ast
import pymongo as pm
from datetime import datetime


### Load connection to database

In [43]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the stanford-cogsci.org user
pswd = auth.values[0][0]
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')

In [44]:
db = conn['devphotodraw_recognition']
coll = db['batched_12afc']

In [45]:
# db = conn['kiddraw']
# coll = db['check_invalid_v5_dev']
# coll.count()

In [46]:
analysis_dir = os.getcwd()
output_dir = os.path.join(analysis_dir,'csv_out')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Only prolific workers have id's that are strings (parsed from url)

In [67]:
# prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': 'batch1_production_june2021' }).distinct('sub_id')
prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': 'batch3_production_june2021' }).distinct('sub_id')
np.size(prolific_workers)
prolific_workers

[u'60c00c32f086c3213de3773',
 u'60c4e2f0a4dbce1286d7d09',
 u'5ea90de5a5bad5377430424',
 u'60c2ac0c6fcf4e382a42f74',
 u'5f2ac32d1f635437876affa',
 u'60c72834098664715c749ca',
 u'5e3a190cc688f023afd6e80',
 u'60c39f9022cd2c6bf7318f5',
 u'5eddce842514f6ae9a1e500',
 u'60c344a48b6bc6a6374ddad']

In [68]:
###### Open up variables for CSV writing
# basic descriptors
subID = []; 
imageName = []
guessed_category = []
drawn_category = [] 

In [69]:
for w in prolific_workers:
    print w
    entries = coll.find({'$and': [{'sub_id': w, 'dataType': 'recognition_rating'}]})
    num_trials =  entries.count()
    print num_trials
    for e in entries:
        subID.append(e['sub_id'])
        imageName.append(e['imageName'])
        guessed_category.append(e['guessed_category'])
        drawn_category.append(e['imageCategory'])

60c00c32f086c3213de3773
144
60c4e2f0a4dbce1286d7d09
99
5ea90de5a5bad5377430424
144
60c2ac0c6fcf4e382a42f74
132
5f2ac32d1f635437876affa
100
60c72834098664715c749ca
93
5e3a190cc688f023afd6e80
31
60c39f9022cd2c6bf7318f5
61
5eddce842514f6ae9a1e500
115
60c344a48b6bc6a6374ddad
22


In [34]:
e

{u'_id': ObjectId('60c7b9cfc0059952db3ba038'),
 u'dataType': u'recognition_rating',
 u'date': u'2021-06-14T20:19:27.393Z',
 u'guessed_category': u'rabbit',
 u'imageCategory': u'rabbit',
 u'imageName': u'https://stanford-cogsci.org:8880/object_drawings/P_rabbit_sketch_age8_CDM_photodraw_e21550689187843.png',
 u'sub_id': u'60c6d6b60fc74b211f17ddf',
 u'version': u'batch1_production_june2021'}

In [64]:

## write out csv 
X_out = pd.DataFrame([subID, imageName, guessed_category, drawn_category])
X_out = X_out.transpose()
X_out.columns = ['subID','imageName', 'guessed_category','drawn_category']
X_out.to_csv(os.path.join(output_dir,'Devphotodraw_recognition_batch2.csv'))


In [62]:
path = 'P_bird_sketch_age5_IPAD5_THU5F3.png'
path = 'P_airplane_sketch_age4_CDM_photodraw_e21547242197593.png'
condition = path.split('/')[-1].split('_')[4]
if path.find('CDM')!=-1:
    site = 'CDM'
else:
    site = 'THU'
site

'CDM'

In [ ]:
# mturk_id
# assignment_id
# trial_id
# hit_id